### Defining Experiment

In [1]:
experiment = {
    
    'experiment_name' : 'TFIDF',
    'collection' : 'keywords',
    'taxonomy_type' : '',
    'papers_type' : '_ENHANCED'
}
keywords = (experiment['collection'] == 'keywords')
shortcut = {'_ENHANCED':'enh', '':'std'}

In [2]:
from libs import *
np.set_printoptions(suppress=True)


EXP_DIR = f"experiments/{experiment['experiment_name']}/{experiment['collection']}"
makedir(EXP_DIR)

### Loading Relevance Matrix

In [3]:
# rel_mat = load_obj(
#     f'input_data/relevance_matrices/{experiment["experiment_name"]}\
# /rel_mat_{experiment["experiment_name"]}\
# _{"keywords_" if keywords else ""}\
# {shortcut[experiment["papers_type"]]}_{shortcut[experiment["taxonomy_type"]]}.pkl'
# )

topics_unique = load_obj(name=
                    f'input_data/taxonomies/unique_topics{experiment["taxonomy_type"]}'
                           )

indices_unique = load_obj(name=
                          f"input_data/taxonomies/unique_indices{experiment['taxonomy_type']}")

papers_df = pd.read_csv(f'input_data/text_collections/papers_df{experiment["papers_type"]}.csv',
                            index_col=0)

if keywords:
    papers_df['keywords'] = papers_df['keywords'].apply(eval)
    texts = papers_df['keywords'].apply(lambda x: ' '.join(x)).str.lower().to_numpy()

### Building Similarity Matrix

In [73]:
# sim_mat = load_obj(name=f'input_data/similarity_matrices/{experiment["experiment_name"]}/'+
# f'sim_mat_{experiment["experiment_name"]}_{"keywords_" if keywords else ""}' + 
# f'{shortcut[experiment["papers_type"]]}_{shortcut[experiment["taxonomy_type"]]}.pkl'

# )

def diag_mult(v, M):
    result = M.copy()
    for i in range(M.shape[0]):
        result[i] = v[i]*result[i]    
    return result

import custom
sim_model = custom.TFIDF(aggregation_F='max', verbose=True)
sim_model.fit(texts, topics_unique)
rel_mat = sim_model.rel_mat
text_pop = rel_mat.sum(axis=1)
rel_mat = rel_mat[(text_pop>0.5)&(text_pop<15), :]
text_pop = rel_mat.sum(axis=1)


inv_pop = 1/text_pop
sim_mat = rel_mat.T@diag_mult(inv_pop, rel_mat)
sim_mat = rel_mat.T@rel_mat


BUILDING TEXT2TOKEN RELEVANCE MATRIX
AGGREGATING TEXT2TOKEN --> TEXT2TOPIC (MAIN RELEVANCE MATRIX)


In [86]:
l, V = np.linalg.eigh(sim_mat)

idx = np.argsort(V[:, 1])

### Clustering 

In [5]:
from sklearn.manifold import spectral_embedding
from sklearn.cluster import KMeans
from fcmeans import FCM
from LaplacianFCM import LaplacianFCM
from sklearn.decomposition import PCA
import utils

clf = LaplacianFCM(n_clusters=12, embedding_dim=12, random_state=1)

clf.fit(sim_mat)

In [6]:
import utils 
reload(utils)
from utils import Clusters

cl_fcm = Clusters(clf.fcm.u.copy(), topics_unique, clf.embedding, random_state=1)
cl_km = Clusters(clf.kmeans.u.copy(), topics_unique, clf.embedding, random_state=1)

### Lifting